# BatchNormalization

https://arxiv.org/abs/1502.03167

### Introduction

2015년 Sergey loffe와 Christian Szegedy는 논문을 통해 BatchNormalization을 제안했다. 2020년 11월 기준으로 22000여회 정도 인용되었으며 Deep Learning Vision 분야에서 가장 많이 사용되는 기법 중의 하나이다. 이 탄생배경을 알기 위해서는 그 때의 상황을 알아볼 필요가 있는데, 한 때 vanishing gradients 문제로 딥러닝은 암흑기를 맞았고 이를 해결하기 위해 많은 연구자들이 다양한 방법들을 시도해보았다. 그 예로 activation function으로 사용되었던 sigmoid의 최대 미분값이 0.2인 것을 문제로 보고 ReLU activation을 제안한 연구, 가중치의 초깃값을 random하게 주는 것이 아닌 xavier initializer, he initializer 등의 방법을 제안한 연구 등이 있겠다. 물론 이러한 방법들이 기존 vanishing gradients 문제를 많이 해결했지만 항상 그 문제가 발생하지 않는다는 보장은 없었다. 그리고 이 해결방안으로 BatchNormalization이 등장했다.

이들은 Gradient의 불안정화가 일어나는 이슈를 'Interval Covariance Shift'라고 주장하고 있는데, 이는 Network의 각 층이나 Activation마다 input의 distribution이 달라지는 현상을 의미한다. 그리고 이 해결방안으로 간단하게 각 층의 input의 distribution을 평균 0, 표준편차 1인 input으로 normalize시키는 방법을 생각해볼 수 있고, 이는 whitening이라는 방법으로 해결할 수 있다(whitening은 기본적으로 들어오는 input의 feature들을 uncorrelated학게 만들어주고 각각의 variance를 1로 만들어주는 작업이다).

논문은 다음과 같은 접근을 취했다.<br>
각각의 feature들이 이미 uncorrelated 되어있다고 가정하고, feature 각각에 대해서만 scalar 형태로 mean과 variance를 구하고 각각 normalization한다. 이 때 training data 전체에 대해 mean과 variance를 구하는 것이 아니라, mini-batch 단위로 접근하여 계산한다. 즉, 현재 택한 mini-batch 안에서만 mean과 variance를 구해서, 이 값을 이용해서 normalization한다. 이후 $\gamma$와 $\beta$를 이용하여 scailing과 shifting을 수행한다.

*discussion*<br>
FC layer에 대해서는 feature별로 위의 접근을 수행한다. 예를 들어, (16,784)인 mini-batch에 대하여(batch_size=16, feature=784) 총 784쌍의 mean, variance, $\gamma$, $\beta$가 사용된다.<br>
반면 Conv layer에 대해서는 channel별로 위의 접근을 수행한다는 점에서 작동 방식이 좀 다르다. 예를 들어, (16, 448, 448, 3)인 mini-batch에 대하여(batch_size=16, height=448, width=448, channel=3) 총 3쌍의 mean, variance, $\gamma$, $\beta$가 사용된다. 이는 CNN에서는 convolution의 성질을 유지하고 싶기 때문에 각 channel을 기준으로 각각의 BatchNormaliztion 변수들을 만든다고 알려져 있다.

### Pseudocode

알고리즘의 개요는 위에 서술한 바와 같다. Neural Network를 학습시킬 때 보통 mini-batch 단위로 데이터를 가져와서 학습을 시키는데, 각 feature별로 mean과 variance를 구해준 다음 normalize 해주고, scale factor와 shift fatcor를 이용해서 새로운 값을 만들어준다. 알고리즘의 개요는 다음과 같다.

<img src="https://i.imgur.com/ALnfYTO.png" width="100%">

실제로 이 Batch Normalization을 네트워크에 적용시킬 때는, 특정 Hidden Layer에 들어가기 전에 BatchNormalization Layer를 더해주어 input을 modify해준 뒤 새로운 값을 activation function으로 넣어주는 방식으로 사용한다(그런데 실제 코드들을 보면 BN과 activation function의 순서가 뒤바뀐 코드들을 종종 보곤 하는데, 학계에 여쭤본 바에 의하면 이 순서는 큰 차이가 없으며 정답은 논문과 같은 형식으로 구현하는 것이다).

<img src="https://i.imgur.com/X4WnIPz.png" width="100%">

### BatchNormalization Inference

Training Data로 학습을 시킬 때는 현재 보고있는 mini-batch에서 mean과 variance를 구하지만, Test Data를 사용하여 Inference를 할 때는 다소 다른 방법을 사용한다. mini-batch의 값들을 이용하는 대신 지금까지 본 전체 데이터를 다 사용한다는 느낌으로, training할 때 현재까지 본 mini-batch들의 이동평균(moving average) 및 unbiased variance estimate의 이동평균을 계산하여 저장해놓은 뒤 이 값으로 normalize를 한다. 마지막에 gamma와 beta를 이용하여 scale/shift 해주는 것은 동일하다. 정리하자면 Training시에는 mini-batch별로 mean과 variance는 계산하고 gamma와 beta는 학습한다. 반면에 Test시에는 training에 계산했던 mean과 variance들을 바탕으로 moving average와 unbiased variance estimation을 구하여 사용하고, 학습했던 gamma와 beta를 그대로 사용한다.

### BatchiNormalization with CNN

참고로 BatchNormalization을 CNN에 적용시킬 때에는 한 가지 더 고려해줘야 할 사항이 있다. conv layer에서는 보통 activation function에 값을 넣기 전 Wx+b 형태로 weight를 적용시키는데, BatchNormalization을 사용하고 싶을 경우 normalization 할 때 beta 값이 b의 역할을 대체할 수 있기 때문에 b를 없애준다.

### Benefit of BatchNormalization

1.기존 Deep Networ에서는 learning rate를 너무 높게 잡을 경우 gradient가 explode/vanish하거나, 나쁜 local minima에 빠지는 문제가 있었다. 이는 parameter들의 scale 때문인데, BatchNormaliztion을 사용할 경우 propagation할 때 parameter의 scale에 영향을 받지 않게 된다. 따라서, learning rate를 크게 잡을 수 있게 되고 이는 빠른 학습을 가능케 한다.

2.BatchNormalization의 경우 자체적인 regularization 효과가 있다. 이는 기존에 사용하던 weight regularization term 등을 제외할 수 있게하며, 나아가 Droput을 제외할 수 있게 한다. Dropout의 효과와 BatchNormalization의 효과가 같기 때문이다. Dropout의 경우 효과는 좋지만 학습 속도가 다소 느려진다는 단점이 있는데, 이를 제거함으로서 학습 속도도 향상된다.

### Example of BN implementation

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(28,28)))
model.add(BatchNormalization())
model.add(Dense(300, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

일반적인 Neural Network에서는 BatchNormalization을 위와 같은 방법으로 사용한다. 이 때 Non-trainable params는 BatchNormalization의 mean과 variance를 의미하며, 이는 학습하는 파라미터가 아닌 계산에 의해 도출되는 값이다. 학습되는 값인 $\gamma$와 $\beta$는 Trainable params이다.

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=(28,28)))
model.add(BatchNormalization())
model.add(Dense(300, kernel_initializer='he_normal', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100, kernel_initializer='he_normal', use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 784)               3136      
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235200    
_________________________________________________________________
batch_normalization_4 (Batch (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               30000     
_________________________________________________________________
batch_normalization_5 (Batch (None, 100)              

CNN에 BatchNormalization을 적용하고 싶은 경우, 앞에서 설명한 내용에 의해 use_bias를 False로 설정한다. 이 때문에 첫 번째 모델과 비교해서 Trainable params: 268,978가 Trainable params: 268,578로 줄어들었음을 확인할 수 있다. 각 conv layer의 bias가 모두 제거된 값이다. 추가적으로 첫 번째 모델은 BN -> Activation 순서로 작성했지만, 두 번째 모델은 Activation -> BN으로 작성했다. 이는 앞에서 설명한 내용처럼 닭이 먼저냐 달걀이 먼저냐와 같은 문제이며, 정답은 적용하고자 하는 논문에서 제시한 방법을 따르는 것이다.

### Conclusion and ML Study Summary

* 각 층에서 활성화 함수를 통과하기 전에나 후에 모델링 연산을 하나 추가
* 이 연산은 단순하게 입력을 원점에 맞추고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터(하나는 스케일 조정에, 다른 하나는 이동에 사용)로 결괏값의 스케일을 조정하고 이동
* 많은 경우 신경망의 첫 번째 층으로 배치 정규화를 추가하면 훈련 세트를 (예를 들면 StandardScaler를 사용하여) 표준화할 필요가 없습니다. 배치 정규화 층이 이런 역할을 대신합니다.
* 그레디언트 소실 문제가 크게 감소하여 하이퍼볼릭 탄젠트나 로지스틱 활성화 함수 같은 수렴성을 가진 활성화 함수를 사용할 수 있습니다.
* 가중치 초기화에 네트워크가 훨씬 덜 민감해집니다.
* 저자들은 훨씬 큰 학습률을 사용하여 학습 과정의 속도를 크게 높일 수 있었습니다.
* 배치 정규화는 규제와 같은 역할을 하여 다른 규제 기법의 필요성을 줄여줍니다.
* 배치 정규화를 사용할 때 에포크마다 더 많은 시간이 걸리므로 훈련이 오히려 느려질 수 있습니다. 하지만 배치 정규화를 사용하면 수렴이 훨씬 빨라지므로 보통 상쇄됩니다. 따라서 더 적은 에포크로 동일한 성능에 도달할 수 있습니다.
---
* 모델의 복잡도를 키웁니다.
* 실행 시간 면에서도 손해입니다. 층마다 추가되는 계산이 신경망의 예측을 느리게 합니다.

* $\gamma$(출력 스케일 벡터)와 $\beta$(출력 이동 벡터)는 일반적인 역전파를 통해 학습됩니다.
* $\mu$(최종 입력 평균 벡터)와 $\sigma$(최종 입력 표준편차 벡터)는 지수 이동 평균을 사용하여 추정됩니다.
* $\mu$와 $\sigma$는 훈련하는 동안 추정되지만 훈련이 끝난 후에 사용됩니다.